In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import warnings
import gc
import joblib 

# [Legacy] 초기 모델링 파이프라인 (Maximalist Approach)
# 특징: 110개 이상의 파생 변수 사용, 수동 검증 셋 구성, 물리적 마스킹(Night Masking) 적용

warnings.filterwarnings('ignore')

BASE_PROCESSED_FILE = 'processed_train.feather'
PROCESSED_TEST_FILE = 'processed_test.feather'
MODEL_FILE = 'final_model.joblib5'
FEATURES_FILE = 'final_features.joblib'
SUBMISSION_FILE = 'submission.csv'
SAMPLE_SUB = 'submission_sample.csv'

# --- 1. 데이터 로드 ---
print(f"[{BASE_PROCESSED_FILE}] 데이터 로드 중...")
df = pd.read_feather(BASE_PROCESSED_FILE)
print("로드 완료.")

# --- 2. 학습 피처 정의 (110개 Full Version) ---
# 초기 전략: 가능한 모든 파생 변수를 투입하여 성능 극대화 시도
features = [
    # 기본 기상 변수
    'ceiling', 'cloud_b', 'precip_1h', 'pressure', 'temp_b', 'uv_idx',
    'vis', 'wind_spd_b', 'cloud_a', 'humidity', 'rain', 'snow',
    'wind_spd_a', 'pv_id', 
    
    # 물리적 파생 변수
    'is_precipitating', 'station_temp_diff',
    'cloud_x_hour', 'uv_idx_x_hour', 'humidity_x_hour', 'temp_x_hour',
    'uv_idx_sq', 'temp_a_sq', 'station_wind_spd_diff', 'station_cloud_diff', 'is_foggy',
    
    # 시차(Lag) 및 이동평균(Rolling) 변수 (다중 윈도우: 15분, 30분, 60분)
    'cloud_a_lag_3', 'cloud_a_rolling_3_mean', 'cloud_a_rolling_3_std',
    'cloud_a_lag_6', 'cloud_a_rolling_6_mean', 'cloud_a_rolling_6_std',
    'cloud_a_lag_12', 'cloud_a_rolling_12_mean', 'cloud_a_rolling_12_std',
    'temp_a_lag_3', 'temp_a_rolling_3_mean', 'temp_a_rolling_3_std',
    'temp_a_lag_6', 'temp_a_rolling_6_mean', 'temp_a_rolling_6_std',
    'temp_a_lag_12', 'temp_a_rolling_12_mean', 'temp_a_rolling_12_std',
    'uv_idx_lag_3', 'uv_idx_rolling_3_mean', 'uv_idx_rolling_3_std',
    'uv_idx_lag_6', 'uv_idx_rolling_6_mean', 'uv_idx_rolling_6_std',
    'uv_idx_lag_12', 'uv_idx_rolling_12_mean', 'uv_idx_rolling_12_std',
    'pressure_lag_3', 'pressure_rolling_3_mean', 'pressure_rolling_3_std',
    'pressure_lag_6', 'pressure_rolling_6_mean', 'pressure_rolling_6_std',
    'pressure_lag_12', 'pressure_rolling_12_mean', 'pressure_rolling_12_std',
    'humidity_lag_3', 'humidity_rolling_3_mean', 'humidity_rolling_3_std',
    'humidity_lag_6', 'humidity_rolling_6_mean', 'humidity_rolling_6_std',
    'humidity_lag_12', 'humidity_rolling_12_mean', 'humidity_rolling_12_std',
    
    # 군집 및 지리 정보 (이중 군집화 적용)
    'location_cluster', 'climate_cluster', 
    
    # 순환 좌표 및 태양 기하학
    'seasonal_hour', 'sun_exposure_factor', 'diurnal_temp_range', 'saturation_deficit',
    'hour_sin', 'hour_cos', 'day_of_year_sin', 'day_of_year_cos', 'hour',
    'daylight_cosine', 'wind_power_a', 'wind_power_b', 'sunset_decimal', 'sunrise_decimal',
    
    # 이상치(Anomaly) 및 평균 대비 편차
    'cloud_anomaly', 'uv_anomaly', 'temp_anomaly',
    'humidity_anomaly', 'vis_anomaly', 'wind_anomaly', 'pressure_anomaly',
    'cluster_hour_humidity_mean', 'cluster_hour_temp_mean',
    'cluster_hour_uv_mean', 'cluster_hour_cloud_mean',
    'cluster_hour_vis_mean', 'cluster_hour_wind_mean', 'cluster_hour_pressure_mean'
]
target = 'nins'

# 실제 데이터프레임에 존재하는 컬럼만 필터링 (안전장치)
features = [f for f in features if f in df.columns]

X_train = df[features]
y_train = df[target]

# 카테고리형 변환
for col in ['pv_id', 'location_cluster', 'climate_cluster']:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype('category')

# --- 3. 밤 시간대 데이터 제거 (Night Masking) ---
print("일출/일몰 맵 기반 '밤 데이터' 학습 제외 처리...")
decimal_hours = df['decimal_hour'].to_numpy()
sunrise_times = df['sunrise_decimal'].to_numpy()
sunset_times = df['sunset_decimal'].to_numpy()

# 결측값 보정 (기본값 적용)
default_sunrise, default_sunset = 4.5, 20.5
sunrise_times = np.where((sunrise_times < -900) | (sunrise_times == 0), default_sunrise, sunrise_times)
sunset_times = np.where((sunset_times < -900) | (sunset_times == 0), default_sunset, sunset_times)

# 마스크 생성 및 적용
train_day_mask = ~((decimal_hours <= sunrise_times) | (decimal_hours >= sunset_times))
X_train = X_train.loc[train_day_mask]
y_train = y_train.loc[train_day_mask]

print(f" -> 학습 데이터: {len(X_train)} 행 (밤 데이터 제외됨)")
del df, decimal_hours, sunrise_times, sunset_times
gc.collect()

# --- 4. 검증 데이터 분리 (Profile Matching Strategy) ---
print("유사 발전소 기반 검증 셋(Validation Set) 분리...")

# Test 셋과 패턴이 유사한 발전소를 사전에 군집을 이용해 수동으로 선정하여 검증에 사용
VALIDATION_PV_IDS = [
    138, 139, 107, 14, 16, 164, 167, 169, 170, 193,
    22, 28, 32, 45, 51, 56, 59, 77, 80, 86, 9
]
# (실제로는 pv_id가 문자열일 수 있으므로 확인 필요, 여기서는 인덱스 매핑 가정)

# pv_id 컬럼이 있다고 가정하고 마스킹
if 'pv_id' in X_train.columns:
    # 실제로는 PV_ID_xx 형태일 수 있으므로 유연하게 처리 필요
    # 여기서는 예시 로직으로 구현
    pass 

# (간소화를 위해 마지막 3개 발전소를 검증용으로 사용)
unique_ids = X_train['pv_id'].unique()
valid_ids = unique_ids[-3:] 
valid_mask = X_train['pv_id'].isin(valid_ids)
train_mask = ~valid_mask

X_train_final = X_train.loc[train_mask]
y_train_final = y_train.loc[train_mask]
X_valid_final = X_train.loc[valid_mask]
y_valid_final = y_train.loc[valid_mask]

print(f" -> Train: {len(X_train_final)}, Valid: {len(X_valid_final)}")

# --- 5. LightGBM 모델 학습 ---
print("LightGBM 모델 학습 시작...")
final_model = lgb.LGBMRegressor(
    objective='mae',
    metric='mae',
    random_state=42,
    n_jobs=-1,
    n_estimators=3000
)

try:
    final_model.fit(
        X_train_final, y_train_final,
        eval_set=[(X_valid_final, y_valid_final)],
        callbacks=[lgb.early_stopping(350, verbose=True)],
        categorical_feature=['location_cluster', 'climate_cluster'] # pv_id 제외 시도
    )
    
    # 모델 및 피처 리스트 저장
    print("모델 저장 중...")
    joblib.dump(final_model, MODEL_FILE)
    joblib.dump(features, FEATURES_FILE)

except Exception as e:
    print(f"[Error] 학습 중 오류 발생: {e}")
    # (메모리 에러 시뮬레이션을 원하면 여기서 raise MemoryError)

# --- 6. 특성 중요도 분석 ---
print("Feature Importance 분석...")
importance_df = pd.DataFrame({
    'feature': features,
    'importance': final_model.feature_importances_
}).sort_values(by='importance', ascending=False)

print(importance_df.head(20))

# --- 7. 예측 및 제출 파일 생성 ---
print(f"[{PROCESSED_TEST_FILE}] 테스트 데이터 로드 및 예측...")
df_test = pd.read_feather(PROCESSED_TEST_FILE)
X_test = df_test[features]

# 카테고리 변환
for col in ['pv_id', 'location_cluster', 'climate_cluster']:
    if col in X_test.columns:
        X_test[col] = X_test[col].astype('category')

preds = final_model.predict(X_test)
preds = np.maximum(preds, 0) # 음수 제거

# 동적 마스크(Dynamic Masking) 적용
# 테스트 데이터 내의 일출/일몰 정보를 활용하여 밤 시간 0 처리
print("동적 마스크(Night Masking) 적용...")
sunrise_vec = df_test['sunrise_decimal'].to_numpy()
sunset_vec = df_test['sunset_decimal'].to_numpy()
decimal_vec = df_test['decimal_hour'].to_numpy()

# 기본값 보정
sunrise_vec = np.where((sunrise_vec < 0) | (sunrise_vec == 0), 4.5, sunrise_vec)
sunset_vec = np.where((sunset_vec < 0) | (sunset_vec == 0), 20.5, sunset_vec)

night_mask = (decimal_vec <= sunrise_vec) | (decimal_vec >= sunset_vec)
preds[night_mask] = 0

# 저장
sub = pd.read_csv(SAMPLE_SUB)
sub['nins'] = preds
sub.to_csv(SUBMISSION_FILE, index=False)
print("제출 파일 생성 완료.")